# autograd

In [ ]:
import torch

x = torch.tensor(2.0, requires_grad=True)
print(x)
y = x**2 + 3*x + 1
print(y)
# backward computes the gradient of y
y.backward()
print(x.grad)

tensor(2., requires_grad=True)
tensor(11., grad_fn=<AddBackward0>)
tensor(7.)


In [24]:
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X, y = data.data, data.target

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
print(X.shape)
print(y.shape)

torch.Size([569, 30])
torch.Size([569, 1])


# a simple nn

In [35]:
from numpy import indices
import torch
import torch.nn as nn

class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(30, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )
        print(self.model)

    def train(self, X, y):
        loss_fn = nn.BCELoss()
        # Adam is an optimization algorithm that can be used instead of the classical stochastic gradient descent procedure 
        # to update network weights iteratively based on training data
        optimizer = optim.Adam(self.model.parameters(), lr=0.001) 
        n_epochs = 32
        batch_size = 10
        for epoch in range(n_epochs):
            for i in range(0, X.size()[0], batch_size):
                Xbatch = X[i:i+batch_size]
                ybatch = y[i:i+batch_size]
                ypred = self.model(Xbatch)
                loss = loss_fn(ypred, ybatch)
                optimizer.zero_grad()
                loss.backward() # computes the gradient of the loss with respect to every parameter in your network that requires gradients
                optimizer.step() # updates the model's parameters (weights and biases) using the gradients computed by loss.backward()
            print(f'Finished epoch {epoch}, latest loss {loss}')
        # compute accuracy (no_grad is optional)
        with torch.no_grad():
            y_pred = self.model(X)
            accuracy = (y_pred.round() == y).float().mean()
            print(f"Accuracy {accuracy}")


net = SimpleNet()
net.train(X, y)

Sequential(
  (0): Linear(in_features=30, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=10, bias=True)
  (3): ReLU()
  (4): Linear(in_features=10, out_features=1, bias=True)
  (5): Sigmoid()
)
Finished epoch 0, latest loss 0.2167855054140091
Finished epoch 1, latest loss 0.20685885846614838
Finished epoch 2, latest loss 0.21720950305461884
Finished epoch 3, latest loss 0.212259441614151
Finished epoch 4, latest loss 0.25669610500335693
Finished epoch 5, latest loss 0.3733963668346405
Finished epoch 6, latest loss 0.7645766735076904
Finished epoch 7, latest loss 1.1019840240478516
Finished epoch 8, latest loss 0.9129229784011841
Finished epoch 9, latest loss 0.4616371691226959
Finished epoch 10, latest loss 0.3853521943092346
Finished epoch 11, latest loss 0.42153820395469666
Finished epoch 12, latest loss 0.7165769338607788
Finished epoch 13, latest loss 0.5446599721908569
Finished epoch 14, latest loss 0.5347124934196472
Finished epoch 15, latest

# using GPU

In [33]:
DEVICE_CPU = torch.device("cpu")
DEVICE_MPS = torch.device("mps")

def get_device():

    if torch.backends.mps.is_available():
        print(f"[get_device] mps device count: {torch.mps.device_count()}")
        result = DEVICE_MPS
    else:
        print("[get_device] MPS device not found!")
        result = DEVICE_CPU
    return result

device = get_device()
print("Using device:", device)

[get_device] mps device count: 1
Using device: mps


In [36]:
net = SimpleNet()
net.to(device)
net.train(X.to(device), y.to(device))

Sequential(
  (0): Linear(in_features=30, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=10, bias=True)
  (3): ReLU()
  (4): Linear(in_features=10, out_features=1, bias=True)
  (5): Sigmoid()
)
Finished epoch 0, latest loss 0.5317256450653076
Finished epoch 1, latest loss 0.40454965829849243
Finished epoch 2, latest loss 0.34670549631118774
Finished epoch 3, latest loss 0.3408992290496826
Finished epoch 4, latest loss 0.35220110416412354
Finished epoch 5, latest loss 0.3650625944137573
Finished epoch 6, latest loss 0.3620116710662842
Finished epoch 7, latest loss 0.3626074492931366
Finished epoch 8, latest loss 0.35720476508140564
Finished epoch 9, latest loss 0.35658517479896545
Finished epoch 10, latest loss 0.34802594780921936
Finished epoch 11, latest loss 0.34165793657302856
Finished epoch 12, latest loss 0.34128135442733765
Finished epoch 13, latest loss 0.3373763859272003
Finished epoch 14, latest loss 0.333152711391449
Finished epoch 15, la